In [ ]:
!pip install "transformers==4.34.0" "datasets[s3]==2.13.0" "sagemaker>=2.190.0" "gradio==3.50.2" --upgrade --quiet

In [ ]:
import sagemaker
import boto3
import os
from sagemaker.huggingface import HuggingFace

In [ ]:
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
# s3 object and client to download and uplaod files
s3 = boto3.resource('s3')
client = s3.meta.client

In [ ]:
# define or get urls and addresses
local_directory = '<LOCAL_PATH_TO_FOLDER_OF_YOUR_IMAGES>' # you can first upload your images into notebook panel and then from there to s3
destination = 'training-diffusion'
bucket = sess.default_bucket()

In [ ]:
# based on above we can set a path in s3 for uploading our images there
training_input_path = f's3://{sess.default_bucket()}/{destination}'

In [ ]:
def upload_folder(local_directory, destination):
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            # construct the full local path
            local_path = os.path.join(root, filename)
            # construct the full Dropbox path
            relative_path = os.path.relpath(local_path, local_directory)
            s3_path = os.path.join(destination, relative_path)
            # relative_path = os.path.relpath(os.path.join(root, filename))
            print('Searching "%s" in "%s"' % (s3_path, bucket))
            try:
                client.head_object(Bucket=bucket, Key=s3_path)
                print("Path found on S3! Skipping %s..." % s3_path)
                # try:
                # client.delete_object(Bucket=bucket, Key=s3_path)
                # except:
                # print("Unable to delete %s..." % s3_path)
            except:
                print("Uploading %s..." % s3_path)
                client.upload_file(local_path, bucket, s3_path)

In [ ]:
# upload images to s3
upload_folder(local_directory, destination)

In [ ]:
# after uploading either you can check by command here or by checking your s3 UI
list(s3.Bucket(bucket).objects.all())

In [ ]:
# There will be some mistakes, especially in the initial attempts when working with S3. If you want to delete unnecessary files in S3:
# s3.Object(bucket, 'key').delete()

In [ ]:
# we will use huggingface estimator so we can pass our args like below since aws doesn't work with action we have changed script or parameters here to handle
hyperparameters = {
    'pretrained_model_name_or_path': "CompVis/stable-diffusion-v1-4",          # pre-trained model
    'instance_data_dir': '/opt/ml/input/data/training',                        # path where sagemaker will save training dataset in s3
    'num_train_epochs': 1,                                                     # number of training epochs
    'class_data_dir': '/opt/ml/input/data/class_data',                         # class images for prior preservation in s3
    'train_batch_size': 1,                                                     # batch size for training
    'gradient_accumulation_steps': 1,                                          # Number of updates steps to accumulate
    'output_dir': '/opt/ml/model',                                             # output directory, where to save assets during training in s3
    'use_8bit_adam': True,                                                     # by setting this we can train dreambooth on ml.g4dn.xlarge 16gb
    'with_prior_preservation': True,                                           # avoid overfitting
    'prior_loss_weight': 1.0,                                                  # weight in loss
    'instance_prompt': "a_photo_of_sks_cat",                                   # underlined justified prompt (will be removed in script args handler)
    'class prompt': "a_photo_of_cat",                                          # underlined justified class prompt
    'resolu800 steps for trainingtion': 512,                                   # resolution
    "lr_scheduler":"constant",                                                 # learning rate scheduler
    'learning_rate': 5e-6,                                                     # learning rate
    "lr_warmup_steps": 0,                                                      # warmup steps
    'num_class_images': 200,                                                   # generate 200 images with class prompt
    'max_train_steps': 800,                                                    # 800 steps for training
}

In [ ]:
huggingface_estimator = HuggingFace(
        entry_point='train_dreambooth.py',        # name of script inside scripts folder
        source_dir='./scripts',                   # folder contains script to train and requirement.txt
        instance_type='ml.g4dn.xlarge',           # instance type selected from training instances
        instance_count=1,                         # number of instances 
        role=role,                                # IAM role which we defined it earlier
        transformers_version = '4.28',            # the transformers version used in the training job
        pytorch_version      = '2.0',             # the pytorch_version version used in the training job
        py_version           = 'py310',           # the python version used in the training job
        disable_output_compression = True         # not compress output to save training time and cost
        hyperparameters = hyperparameters
)

In [ ]:
huggingface_estimator.fit({'training': training_input_path})